<a href="https://colab.research.google.com/github/chrischiancone/AI/blob/main/Cold_Call_Assist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install pandas
!pip install requests

In [24]:
import requests
import pandas as pd

# Define your API key and headers for authentication
api_key = 'sk-khkx1j9clzm2otroaucxvjhfiqek3dmtu6bnu65vlk8gaksyj25h5neivu8slvg269'
headers = {
    "authorization": api_key,
    "Content-Type": "application/json"
}

# Load the call list from a CSV file
df = pd.read_csv('/content/drive/MyDrive/AI/Numbers.csv')


# Define the payload for the batch call
batch_payload = {
    "base_prompt": """You're calling to speak with {{name}} about thier outstanding cour fines.
                  Tell them that it is important to pay thier fines to avoid recovery actions.
                  You're calling from The City of Carrollton. If the user wants to perform payment, transfer to {{default}}.
                   """,
    "call_data": [
        {
            "phone_number": str(row['Phone Number']),
            "name": row['Name'],
        } for index, row in df.iterrows()
    ],
    #define transfer phone numbers
    "transfer_list": {
        "default": "19728327095",
        "sales": "1555552323"
    },
    #Define tough to pronounce words
    "pronunciation_guide": [
      {
        "word": "realty",
        "pronunciation": "real-tea",
      },
      {
        "word": "Estate",
        "pronunciation": "Es-state",
      }
    ],
    "record": True,
    "voicemail_message": "Hello, this is Jim calling from the City of Carrollton. I'm reaching out because we've noticed there's an outstanding court payment associated with your account. We'd like to help you resolve this as soon as possible. If you'd like to speak with a court administrator, please give us a call back at your earliest convenience. You can also say 'transfer' during the call to be connected directly to a court representative. Thank you, and have a great day.",
    "model": "enhanced",
    "wait_for_greeting": False,
    "first_sentence": """Hello,,, I'm Jim,,, calling from the City of Carrollton. We've noticed that there's an outstanding
                         court payment on your account. Would you like to speak with a court administrator to resolve this today?
                         At any time, you can say 'transfer' to connect with a court representative.""",
    "voice": "matt",            #this changed on Bland AI's side, so I updated here - different from video
    "reduce_latency": True,
    "temperature": 0.7,
    "test_mode": False,
    "campaign_id": "Courts Recovery",
    "label": "Courts Recovery"
}

# Sending the batch of calls
send_batch_url = "https://api.bland.ai/v1/batches"
response = requests.post(send_batch_url, json=batch_payload, headers=headers)
batch_response = response.json()

if batch_response.get('message') == 'success':
    print("Batch sent successfully")
    batch_id = batch_response.get('batch_id')
else:
    print("Failed to send batch")
    print(response.text)
    # If the batch fails to send, halt further execution
    raise Exception("Batch sending failed, halting execution.")

Batch sent successfully


In [25]:
# Analyze the batch calls and give a status of each
analyze_url = f"https://api.bland.ai/v1/batches/{batch_id}/analyze"
analyze_payload = {
    "goal": "Is customer interested in paying thier court fines",
    "questions": [
        ["Who answered the call?", "human or voicemail"],
        ["Was the customer interested in paying?", "yes or no"],
        ["Did they ask to be transferred?", "yes or no"],
        ["Was the customer interested in payment plan?", "string"]
    ]
}

analyze_response = requests.post(analyze_url, json=analyze_payload, headers=headers)
analyze_result = analyze_response.json()

if analyze_result['status'] == 'success':
    print("Batch analyzed successfully")
    answers = analyze_result['answers']
    # Compile analysis results into a report
    report_data = []
    for call_id, analysis in answers.items():
        report_row = {
            "Call ID": call_id,
            "Answered By": analysis[0] if analysis[0] is not None else "N/A",
            "Interested in Paying": analysis[1] if analysis[1] is not None else "N/A",
            "Ask to be transferred": analysis[2] if analysis[2] is not None else "N/A",
            "Interested in Payment Plan": analysis[3] if analysis[3] is not None else "N/A"
        }
        report_data.append(report_row)

    # Convert report data into a DataFrame
    report_df = pd.DataFrame(report_data)

    # Use to_string for better alignment in the Colab output window, index=False removes the DataFrame index
    print(report_df.to_string(index=False))
        # Optionally save to CSV
    #report_df.to_csv('call_outcomes_report.csv', index=False)
else:
    print("Failed to analyze batch")
    print(analyze_response.text)

Batch analyzed successfully
                             Call ID Answered By Interested in Paying Ask to be transferred Interested in Payment Plan
8487da4a-316c-4951-8f1d-548b43b2f6c5         N/A                  N/A                   N/A                        N/A
6f1487db-39c9-4637-bd0c-86ad13eec877       human                  yes                   yes               payment plan


In [28]:
# Use stop_call(call_id) function where necessary to stop calls
url = "https://api.bland.ai/v1/calls/{call_id}/stop"

In [29]:
url = "https://api.bland.ai/v1/calls/8487da4a-316c-4951-8f1d-548b43b2f6c5/stop"
